In [1]:
# check langchain version it should be 0.3 if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.1.20
0.0.38


In [5]:
# -----------------------------------------------------------
# Chain of Thought Reasoning System
# Demonstrates step-by-step reasoning for complex problem solving
# using an LLM via OpenAI API.
# -----------------------------------------------------------

# Initialize client
import os
import json
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configure llm via .env

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [7]:
# Define reasoning templates for different problem types
REASONING_TEMPLATES = {
    "math_word_problem": """
Let's think through this step by step:
1. First, identify what the problem is asking for
2. Extract the relevant numbers and variables
3. Determine the mathematical operations needed
4. Set up the equation or calculation
5. Solve step by step
6. Verify the answer makes sense
""",

    "logical_deduction": """
Let's reason through this logically:
1. List all the given facts and constraints
2. Identify relationships between elements
3. Look for contradictions or implications
4. Make inferences from the available information
5. Build conclusions step by step
6. Check for consistency
""",

    "code_debugging": """
Let's debug this systematically:
1. Understand what the code should do vs what it actually does
2. Identify the specific error or unexpected behavior
3. Trace through the code execution step by step
4. Check variable values and data flow
5. Isolate the problematic section
6. Propose and test fixes
"""
}


In [8]:
# Example problems - you can replace these dynamically
problems = {
    "math_problem": "If a store has 15 apples and sells 8, then receives a shipment of 12 more apples, how many apples do they have now?",
    
    "logic_puzzle": "There are three people: Alice, Bob, and Charlie. Alice is taller than Bob. Charlie is shorter than Bob. Who is the tallest?",
    
    "debugging_issue": "This Python function should return the sum of even numbers in a list, but it's returning incorrect results:\n\n```python\ndef sum_even_numbers(numbers):\n    total = 0\n    for num in numbers:\n        if num % 2 == 0:\n            total += num\n    return total\n\n# Test case: sum_even_numbers([1, 2, 3, 4, 5, 6]) returns 9 instead of 12\n```"
}


In [9]:
# Select a problem to solve
selected_problem = problems["math_problem"]
problem_type = "math_word_problem"


In [10]:
# Build the chain of thought prompt
prompt = f"""
You are an AI reasoning assistant. Your task is to solve problems using clear, step-by-step chain of thought reasoning.

[PROBLEM]
{selected_problem}

[REASONING APPROACH]
{REASONING_TEMPLATES[problem_type]}

Please work through the problem systematically, showing your reasoning at each step before providing the final answer.

Format your response as:
Step 1: [Your first reasoning step]
Step 2: [Your next reasoning step]
...
Final Answer: [Your conclusive answer]
"""


In [15]:
from langchain.schema import SystemMessage, HumanMessage

messages = [
    SystemMessage(content=(
        "You are a clear, careful assistant. Do NOT reveal private chain-of-thought. "
        "Provide a concise final answer and a brief, high-level, non-sensitive stepwise summary "
        "explaining the approach (3–8 numbered steps). Keep the internal detailed reasoning private."
    )),
    HumanMessage(content=prompt),
]

# Call the model
response = llm(messages)  # returns a ChatMessage object
result_text = response.content

# Print result
print("\n--- MODEL OUTPUT ---\n")
print(result_text)


--- MODEL OUTPUT ---

Step 1: Identify what the problem is asking for. The problem is asking for the total number of apples the store has after selling some and receiving a shipment.

Step 2: Extract the relevant numbers. The store starts with 15 apples, sells 8 apples, and then receives 12 more apples.

Step 3: Determine the mathematical operations needed. We need to subtract the number of apples sold from the initial amount and then add the number of apples received.

Step 4: Set up the calculation. The calculation can be expressed as: 
Total apples = Initial apples - Sold apples + Received apples
Total apples = 15 - 8 + 12

Step 5: Solve step by step. 
- First, calculate the apples after selling: 15 - 8 = 7
- Then, add the received apples: 7 + 12 = 19

Step 6: Verify the answer makes sense. Starting with 15 apples, selling 8 leaves 7, and adding 12 results in 19 apples, which is logical.

Final Answer: 19 apples


# Alternative prompt for more complex multi-step reasoning


In [ ]:
complex_prompt = f"""
You are an expert problem solver. Use chain of thought reasoning to break down this problem into manageable steps.

Problem: {selected_problem}

Think through this carefully:

First, let me understand what's being asked:
[Your initial analysis]

Now, let me identify the key information:
[Extract relevant facts and data]

Next, I'll determine the approach:
[Outline your solution strategy]

Then, I'll work through the steps:
[Detailed step-by-step reasoning]

Finally, I'll verify the solution:
[Check your work and confirm]

Answer:
"""

In [18]:
from langchain.schema import SystemMessage, HumanMessage

messages = [
    SystemMessage(content=(
        "You are a clear, careful assistant. Do NOT reveal private chain-of-thought. "
        "Provide a concise final answer and a brief, high-level, non-sensitive stepwise summary "
        "explaining the approach (3–8 numbered steps). Keep the internal detailed reasoning private."
    )),
    HumanMessage(content=complex_prompt),
]

# Call the model
complex_response = llm(messages)  # returns a ChatMessage object
result_text = complex_response.content

# Print result
print("\n--- MODEL OUTPUT ---\n")
print(result_text)


--- MODEL OUTPUT ---

Let's break down the problem step by step.

### Initial Analysis
The problem asks for the total number of apples a store has after selling some and then receiving a shipment. We need to account for both the apples sold and the apples received.

### Key Information
1. The store starts with **15 apples**.
2. The store sells **8 apples**.
3. The store receives a shipment of **12 apples**.

### Approach
To find the total number of apples the store has now, we can follow these steps:
1. Subtract the number of apples sold from the initial amount.
2. Add the number of apples received from the shipment to the result from step 1.

### Step-by-Step Reasoning
1. **Start with the initial number of apples**: 
   - Initial apples = 15

2. **Subtract the apples sold**: 
   - Apples after selling = Initial apples - Apples sold
   - Apples after selling = 15 - 8 = 7

3. **Add the apples received from the shipment**: 
   - Total apples now = Apples after selling + Apples received


TO DO: Now try with this SystemMessage(content=: replace below and observe output
 
"You are a meticulous reasoning assistant that shows your work step by step."

In [20]:
from langchain.schema import SystemMessage, HumanMessage

def reflective_reasoning(problem: str, problem_type: str, llm, temp: float = 0.1, max_tokens: int = 1000):
    """
    Ask the model for a safe, structured explanation + reflection.
    Returns a dict with keys: final_answer, steps_summary, verification, raw_output.
    """
    # System message: explicitly instruct the model NOT to reveal private chain-of-thought
    system = SystemMessage(content=(
        "You are a careful, clear assistant. Do NOT reveal internal/private chain-of-thought. "
        "Provide a short 'Final Answer', a numbered 'Steps (summary)' that explains the approach at a high level "
        "(3-8 concise steps), and a 'Reflection & Verification' section that checks the result and lists simple checks."
    ))

    user_prompt = f"""Problem type: {problem_type}

Problem:
{problem}

Please respond in this exact format:

Final Answer:
- <one or two sentence conclusion>

Steps (summary):
1) ...
2) ...
(keep to 3-8 short numbered steps describing the method — not private thoughts)

Reflection & Verification:
- Brief checks you did or simple ways a learner can verify the answer
- Any assumptions made
"""

    human = HumanMessage(content=user_prompt)

    # call the llm
    response = llm([system, human], temperature=temp, max_tokens=max_tokens)
    raw = response.content.strip()

    # naive parsing: split by section headers
    # (keeps learners' code simple — robust parsing can be added later)
    def extract_section(text, header):
        idx = text.find(header)
        if idx == -1:
            return ""
        # find next header (either "Steps (summary):" or "Reflection & Verification:" or end)
        next_headers = ["Final Answer:", "Steps (summary):", "Reflection & Verification:"]
        # remove current header from search list
        remaining = [h for h in next_headers if h != header]
        next_idx = len(text)
        for h in remaining:
            i = text.find(h, idx + len(header))
            if i != -1 and i < next_idx:
                next_idx = i
        return text[idx + len(header):next_idx].strip()

    final_answer = extract_section(raw, "Final Answer:")
    steps_summary = extract_section(raw, "Steps (summary):")
    verification = extract_section(raw, "Reflection & Verification:")

    return {
        "final_answer": final_answer,
        "steps_summary": steps_summary,
        "verification": verification,
        "raw_output": raw
    }


# Example usage
selected_problem = "A bicyclist travels 30 km at 15 km/h and then 20 km at 10 km/h. What is the average speed for the whole trip?"
problem_type = "algebra / average speed"

result = reflective_reasoning(selected_problem, problem_type, llm)
print("\nFINAL ANSWER:\n", result["final_answer"])
print("\nSTEPS (SUMMARY):\n", result["steps_summary"])
print("\nREFLECTION & VERIFICATION:\n", result["verification"])




FINAL ANSWER:
 - The average speed for the whole trip is 12 km/h.

STEPS (SUMMARY):
 1) Calculate the time taken for the first part of the trip (30 km at 15 km/h).
2) Calculate the time taken for the second part of the trip (20 km at 10 km/h).
3) Find the total distance traveled by adding both distances.
4) Find the total time taken by adding both times.
5) Calculate the average speed using the formula: average speed = total distance / total time.

REFLECTION & VERIFICATION:
 - The calculations for time taken for each segment were checked by using the formula time = distance/speed.
- The total distance and total time were summed correctly.
- The average speed was calculated correctly using the total distance and total time.
- Assumptions made include that the speeds were constant during each segment of the trip.


In [21]:
# If you want the full raw model response (for debugging)
result["raw_output"]


'Final Answer:\n- The average speed for the whole trip is 12 km/h.\n\nSteps (summary):\n1) Calculate the time taken for the first part of the trip (30 km at 15 km/h).\n2) Calculate the time taken for the second part of the trip (20 km at 10 km/h).\n3) Find the total distance traveled by adding both distances.\n4) Find the total time taken by adding both times.\n5) Calculate the average speed using the formula: average speed = total distance / total time.\n\nReflection & Verification:\n- The calculations for time taken for each segment were checked by using the formula time = distance/speed.\n- The total distance and total time were summed correctly.\n- The average speed was calculated correctly using the total distance and total time.\n- Assumptions made include that the speeds were constant during each segment of the trip.'

In [ ]:
# Key Features:
# Multiple Reasoning Templates - Different approaches for math, logic, and debugging problems
# Structured Step-by-Step Format - Clear progression through reasoning steps
# Self-Reflection Capability - Includes verification and alternative approach consideration
# Multi-Problem Processing - Can handle sequences of problems with appropriate reasoning styles
# Flexible Prompt Construction - Easy to adapt for different problem types

# Benefits of Chain of Thought:
# Transparent Reasoning - Shows how the solution was reached
# Error Identification - Makes it easier to spot where reasoning goes wrong
# Learning Tool - Demonstrates problem-solving methodology
# Better Accuracy - Step-by-step reasoning often leads to more correct answers
# Debugging Friendly - Easy to follow the logic trail